## Сбор списка всех услуга#код за временной промежуток

In [1]:
import sys 
import os
import re
import json
import time
import pandas as pd
from datetime import date, timedelta
from typing import Union, Callable
from functools import wraps

import base64
import requests
from requests.auth import HTTPBasicAuth

In [2]:
project_path = os.path.abspath('..')
if project_path not in sys.path:
    sys.path.append(project_path)
sys.path

['C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner\\src',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\python311.zip',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\DLLs',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3',
 '',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner']

In [3]:
from config.config import config
from src.logger import logger
from dotenv import load_dotenv

load_dotenv()

CONFIG INFO:
sys.frozen: False
magick_opt: ['-colorspace', 'Gray', '-quality', '100', '-units', 'PixelsPerInch', '-density', '350']
NAME_scanned: 0_scan
NAME_text: 1_text
NAME_verified: EXPORT
BASE_DIR: C:\Users\Filipp\PycharmProjects\Invoice_scanner
POPPLER_PATH: C:\Program Files\poppler-24.07.0\Library\bin
magick_exe: magick
server_datas: \\10.10.0.3\docs\Transfer\Filipp\1_shared\invoice_scanner
CONFIG: C:\Users\Filipp\PycharmProjects\Invoice_scanner\config
IN_FOLDER: C:\Users\Filipp\PycharmProjects\Invoice_scanner\IN
EDITED: C:\Users\Filipp\PycharmProjects\Invoice_scanner\EDITED
CHECK_FOLDER: C:\Users\Filipp\PycharmProjects\Invoice_scanner\CHECK
CSS_PATH: ../../../../config/styles.css
JS_PATH: ../../../../config/scripts.js
crypto_env: C:\Users\Filipp\PycharmProjects\Invoice_scanner\config\encrypted.env
TESTFILE: C:\Users\Filipp\PycharmProjects\Invoice_scanner\config\test.json
GPTMODEL: gpt-4o-2024-08-06
chroma_path: C:\Users\Filipp\PycharmProjects\Invoice_scanner\config\chroma
embed

True

In [4]:
config['user_1C'] = os.getenv('user_1C')
config['password_1C'] = os.getenv('password_1C')

auth = HTTPBasicAuth(config['user_1C'], config['password_1C'])

In [5]:
def cache_http_requests(func):
    """ Декоратор для кэширования запросов на основе URL """

    cache = {}
    max_cache_size = 40

    @wraps(func)
    def wrapper(function, *args, **kwargs):
        # Формируем ключ кэша из функции + "_" + аргументы
        function_args = r'_'.join(args)
        url_cache_key = function + r'_' + function_args

        # Проверяем, есть ли результат в кэше для данного URL
        if url_cache_key in cache:
            logger.print("Получение результата из кэша...")
            return cache[url_cache_key]

        # Выполняем запрос и сохраняем результат в кэше
        result = func(function, *args, **kwargs)
        cache[url_cache_key] = result

        if len(cache) > max_cache_size:
            cache.pop(next(iter(cache)))

        return result

    return wrapper


@cache_http_requests
def cup_http_request(function, *args, kappa=False, encode_off=False) -> Union[list, dict, None]:
    user_1C = config['user_1C']
    password_1C = config['password_1C']

    # Определение серверов
    if kappa:
        primary_base = r'http://kappa5.group.ru:81/ca/hs/interaction/'
        secondary_base = r'http://10.10.0.10:81/ca/hs/interaction/'
    else:
        primary_base = r'http://10.10.0.10:81/ca/hs/interaction/'
        secondary_base = r'http://kappa5.group.ru:81/ca/hs/interaction/'

    if encode_off:
        encode_func: Callable = lambda x: x
    else:
        encode_func: Callable = lambda x: base64.urlsafe_b64encode(x.encode()).decode()

    function_args = r'/'.join(map(encode_func, args))

    try:
        # Формируем URL для первого сервера
        primary_url = primary_base + function + r'/' + function_args
        logger.print(f"Попытка запроса: {primary_url}")

        # Попытка отправить запрос на первый сервер
        response = requests.get(primary_url, auth=HTTPBasicAuth(user_1C, password_1C))

        # Если первый запрос успешен, возвращаем результат
        if response.status_code == 200:
            return response.json()
        else:
            logger.print(f"Ошибка при запросе к первому серверу: {response.status_code} - {response.reason}")
    except Exception as error:
        logger.print(error)

    try:
        # Формируем URL для второго сервера
        secondary_url = secondary_base + function + r'/' + function_args
        logger.print(f"Попытка запроса ко второму серверу: {secondary_url}")

        # Попытка отправить запрос на второй сервер
        response = requests.get(secondary_url, auth=HTTPBasicAuth(user_1C, password_1C))

        # Возвращаем результат, если успешен
        if response.status_code == 200:
            return response.json()
        else:
            logger.print(f"Ошибка при запросе ко второму серверу: {response.status_code} - {response.reason}")
            return None
    except Exception as error:
        logger.print(error)
        return None


def add_partner(response: Union[list, dict, None]):
    if not isinstance(response, list):
        return response

    regex = r'(.*) (от) (.*)'
    matches = [re.fullmatch(regex, deal, re.IGNORECASE) for deal in response]
    if all(matches):
        deals = [match.group(1) for match in matches]
        partners = [cup_http_request(r'ValueByTransactionNumber', deal, 'Контрагент') for deal in deals]
        partners = [p[0] if p else '' for p in partners]

        new_response = []
        for m, p in zip(matches, partners):
            deal_and_partner = m.group() + f" | {p}"
            new_response.append(deal_and_partner)
        return new_response

    else:
        return response


def cup_http_request_partner(function, *args, kappa=False, encode_off=False) -> Union[list, dict, None]:
    response = cup_http_request(function, *args, kappa=kappa, encode_off=encode_off)
    return add_partner(response)

## get services 

In [6]:
# Пример работы функции

res = cup_http_request("InvoicesItemsByDates", '01-10-2024', '10-10-2024', encode_off=True)

print(len(res))
res[0:3]

Попытка запроса: http://10.10.0.10:81/ca/hs/interaction/InvoicesItemsByDates/01-10-2024/10-10-2024
275


[{'Услуга': 'Сверхнормативное использование автотранспорта',
  'УслугаКод': 'ТК-003329',
  'Упоминаний': 2,
  'Счетов': 1},
 {'Услуга': 'Погрузка ручная ',
  'УслугаКод': 'ТК-003336',
  'Упоминаний': 1,
  'Счетов': 1},
 {'Услуга': 'Взвешивание',
  'УслугаКод': 'ТК-006728',
  'Упоминаний': 13,
  'Счетов': 9}]

In [20]:
# Получаем первую и последнюю дату каждого месяца

def first_and_last_day(year) -> list[tuple[str, str]]:
    result = []
    for month in range(1, 13):
        first_day = date(year, month, 1)
        if month == 12:
            last_day = date(year + 1, 1, 1) - timedelta(days=1)
        else:
            last_day = date(year, month + 1, 1) - timedelta(days=1)
        result.append((first_day.strftime('%d-%m-%Y'), last_day.strftime('%d-%m-%Y')))
    return result

year = 2024
days = first_and_last_day(year)
for i in days:
    print(i)

('01-01-2024', '31-01-2024')
('01-02-2024', '29-02-2024')
('01-03-2024', '31-03-2024')
('01-04-2024', '30-04-2024')
('01-05-2024', '31-05-2024')
('01-06-2024', '30-06-2024')
('01-07-2024', '31-07-2024')
('01-08-2024', '31-08-2024')
('01-09-2024', '30-09-2024')
('01-10-2024', '31-10-2024')
('01-11-2024', '30-11-2024')
('01-12-2024', '31-12-2024')


In [21]:
# Вызываем InvoicesItemsByDates для каждого месяца. Сохраняем все словари в список lst

lst = []
for i, j in days:
    lst.extend(cup_http_request("InvoicesItemsByDates", i, j, encode_off=True))

print(len(lst))
print(lst[0:3])

Попытка запроса: http://10.10.0.10:81/ca/hs/interaction/InvoicesItemsByDates/01-01-2024/31-01-2024
Попытка запроса: http://10.10.0.10:81/ca/hs/interaction/InvoicesItemsByDates/01-02-2024/29-02-2024
Попытка запроса: http://10.10.0.10:81/ca/hs/interaction/InvoicesItemsByDates/01-03-2024/31-03-2024
Попытка запроса: http://10.10.0.10:81/ca/hs/interaction/InvoicesItemsByDates/01-04-2024/30-04-2024
Попытка запроса: http://10.10.0.10:81/ca/hs/interaction/InvoicesItemsByDates/01-05-2024/31-05-2024
Попытка запроса: http://10.10.0.10:81/ca/hs/interaction/InvoicesItemsByDates/01-06-2024/30-06-2024
Попытка запроса: http://10.10.0.10:81/ca/hs/interaction/InvoicesItemsByDates/01-07-2024/31-07-2024
Попытка запроса: http://10.10.0.10:81/ca/hs/interaction/InvoicesItemsByDates/01-08-2024/31-08-2024
Попытка запроса: http://10.10.0.10:81/ca/hs/interaction/InvoicesItemsByDates/01-09-2024/30-09-2024
Попытка запроса: http://10.10.0.10:81/ca/hs/interaction/InvoicesItemsByDates/01-10-2024/31-10-2024
Попытка за

In [22]:
# подготовляваем данные для формирования DataFrame

services = []
codes = []
num_docs = []

for i in lst:
    services.append(i['Услуга'])
    codes.append(i['УслугаКод'])
    num_docs.append(i['Счетов'])

In [23]:
# формируем DataFrame

df = pd.DataFrame(zip(services, codes, num_docs))
df.columns = ['service', 'code', 'num']
df.head()

,service,code,num
0,Ксерокопирование комплекта документов,ТК-003284,1
1,Хранение груженых контейнеров,ТК-003384,3
2,Линейный сбор,Т1389,2
3,Хранение груза на сухом складе,ТК-006721,3
4,Взвешивание,ТК-006728,26


### Если для одного service встречается несколько code (берем самый часто встречаемый code)

In [24]:
# группируем по service, code с суммированием кол-ва встречаемости

gr = df.groupby(['service', 'code']).agg(sum_num=('num', 'sum')).reset_index()
gr.head()

,service,code,sum_num
0,20-футовый контейнер - перевалка груженого кон...,ТК-009698,1
1,20-футовый контейнер - перемещение груженого к...,ТК-009699,1
2,ADDITIONAL,ТК-008437,10
3,AGENT FEE (МОРЕ),ТК-008580,182
4,AMENDMENT FEE,ТК-008738,8


In [25]:
# сортируем по встречаемости

gr = gr.sort_values('sum_num', ascending=False)
gr.head()

,service,code,sum_num
526,Организация автотранспортной перевозки,000000190,4199
662,Оформление документации,Т0778,1964
925,"Просмотр для выявления семян сорных растений, ...",ТК-008928,1921
920,"Просмотр для выявления семян сорных растений, ...",ТК-008932,1899
1068,Срочное выполнение работ,ТК-009054,1734


In [26]:
# формируем итоговый список услуг (в него не попадут повторяющиеся service+code)

d = {}

for idx, row in gr.iterrows():
    service = row['service']
    code = row['code']
    if service not in d:
        d[service.strip()] = code.strip()

# d = {'service1': code1, 'service2': code2 ...}

In [27]:
len(d)

1371

In [28]:
# create new all_services.json

# with open(os.path.join(config['all_services_file']), 'w', encoding='utf-8') as file:
#     json.dump(d, file, ensure_ascii=False, indent=4)

In [29]:
# update existing all_services.json

with open(os.path.join(config['all_services_file']), 'r', encoding='utf-8') as file:
    old = json.load(file)

print(len(old))

old.update(d)

print(len(old))


with open(os.path.join(config['all_services_file']), 'w', encoding='utf-8') as file:
    json.dump(old, file, ensure_ascii=False, indent=4)

1432
1432
